### Functions

In [9]:
from itertools import product, combinations_with_replacement, combinations
import numpy as np
from scipy import stats
from typing import List, Callable
from math import sqrt

In [ ]:
def prob(
    k: int,  # number of null hypotheses rejected by the BH procedure
    j: int,  # number of false rejections
    m: int,  # number of total hypotheses
    n: int,  # number of truly null hypotheses
    alt_cdf: Callable,  # distribution of p-values under alternative
    α: float,
) -> float:
    # Returns the probability of observing k & j given fdr, m, n, and alt_cdf
    # Defined in DOI: 10.2202/1557-4679.1103, eq. (5)
    # Pr(J=j, K=k) is based on a specific sum the joint CDFs of the order statistics Φ
    assert m > n

    if j > k:
        return 0

    probability = 0

    for combo in eq_5_combinations(m, k):
        sign = -1 if len(combo) % 2 else 1  # alternate adding and subtracting

        probability += sign * F_ordinal(combo, j, m, n, alt_cdf, α)

    return probability


def eq_5_combinations(m: int, k: int) -> List[List[int]]:
    # Defined in DOI: 10.2202/1557-4679.1103, eq. (5)
    # returns a list of ordered lists with unique integer elements representing order statistics

    # (k),
    # (combinations of (k) and another order statistic (r) which is k < r <=m)
    # (combinations of (k) and two other order statistic (r) (s) which are k < r < s <=m)
    # (combinations of (k) and three other order statistic (r)(s)(t) which are k < r < s < t <=m)
    # ...
    # (k, ..., m)
    #
    # for k = 1, m = 3, returns [[1], [1, 2], [1, 3], [1, 2, 3]]

    for sum_idx in range(0, m - k + 1):  # first sum, second sum, etc.
        for combo in combinations(range(k + 1, m + 1), r=sum_idx):
            yield [k, *combo]


def F_ordinal(combo, j, m, n, alt_cdf, α):
    # Defined in DOI: 10.2202/1557-4679.1103, eq. (12)

    total = 0
    b = _b(m, α)

    # pn0 = 0 and pne+1 = 1
    p = (0, *[b(p) for p in combo], 1)

    for i_vec in _I(combo=combo, m=m):
        for μ in _M(combo, i_vec, n_row=m, true_null=n, false_rejections=j):
            for h in range(len(combo) + 1):
                subtotal = 1
                for i in np.nonzero(μ[:, h])[0] + 1:  # find i s.t. μ_{ih}=1
                    F = _F(i, n, alt_cdf)

                    subtotal *= F(p[h + 1]) - F(p[h])
                total += subtotal
                if total > 1:
                    breakpoint()

    if not (0 <= total <= 1):
        breakpoint()

    return total


def _b(m, alpha) -> Callable:
    # BH rejection bounds
    def b(i):
        return (i * alpha) / m

    return b


def _F(i, n, alt_cdf) -> Callable:
    # p-value's cdf
    # uniform when it's a true null

    if i <= n:
        return lambda p: p
    else:
        return alt_cdf


# I index set, =[i_vec_0,...,i_vec_e+1]
def _I(combo, m):
    I_vec = combinations_with_replacement(range(m + 1), len(combo) + 2)
    # i_vec_0=0, i_vec_e+1=m
    I_vec = filter(lambda x: x[0] == 0 and x[len(combo) + 1] == m, I_vec)
    # non-descending order
    I_vec = filter(lambda x: sorted(x) == list(x), I_vec)
    # i_h>=n_h for h=1,...,len(perm)
    for i in filter(
        lambda x: all(i >= combo[y] for y, i in enumerate(x[1 : len(combo) + 1])), I_vec
    ):
        yield i


def _M(combo: List[int], i_vec: List[int], n_row: int, true_null, false_rejections):
    # M is set of μ; μ's size=[m,e+1]
    # i_vec is a vector in I(perm)
    assert n_row >= true_null
    assert false_rejections <= true_null

    n_col = len(combo) + 1

    # constraint 11
    possible_rows = np.identity(n_col)
    candidates = [np.array(a) for a in product(possible_rows, repeat=n_row)]

    for arr in candidates:
        # constaint 9
        if np.sum(arr[0:true_null, 0]) == false_rejections:
            # constraint 10
            if np.all(
                [
                    np.sum(arr[::, h]) == (i_vec[h + 1] - i_vec[h])
                    for h in range(1, n_col)
                ]
            ):
                yield arr




### Calculations

In [ ]:
# parameters from table 1
ALPHA = 0.05
M = 2
N = 1


def alt_cdf(p):
    # CDF of p-values under with H0: δ = 0 when actually δ = 1, σ = 1
    SIZE = 100
    DELTA = 1

    Z = stats.norm.ppf(p / 2, scale=1 / sqrt(SIZE))
    return stats.norm.cdf(Z, loc=DELTA, scale=1 / 1 / sqrt(SIZE))


expectation = 0

for k in range(1, M + 1):
    for j in range(0, N + 1):
        possibility = (k - j) / (M - N)
        probability = prob(k, j, M, N, alt_cdf, ALPHA)

        expectation += possibility * probability
        if not (0 <= probability <= 1):
            print(k, j, probability)

In [52]:
# parameters from table 1
delt = 1
var = 1
alpha = 0.05
s = 5
M = 5
N = 3
null_test=[1,2,3]

In [ ]:
K = np.arange(1, M + 1)
J = np.arange(N)
false_null = M - N
avg_pow = 0

# formula(14)
for [k, j] in np.array(np.meshgrid(K, J)).T.reshape(-1, 2):
    if k <= j:
        continue
    print(k,j,(k - j) / false_null * prob(k, j,fdr=alpha,m=M,n=N,delta=delt,variance=var,size=s, null=null_test))
    avg_pow += (k - j) / false_null * prob(k, j, fdr=alpha,m=M,n=N,delta=delt,variance=var,size=s, null=null_test)

print(avg_pow)

### Bug

In [53]:
prob(k=1, j=1, fdr=alpha,m=M,n=N,delta=delt,variance=var,size=s, null=null_test)

AssertionError: invalid when perm=(1,), k=1,j=1

In [56]:
f_ordinal((1,), k, j, M, N, alpha, null_test, delt,var,s)

3.970299

In [58]:
F(1, b(perm=(1,), fdr=alpha,m=M)[2],null=null_test,fdr=alpha,delta=delt,variance=
                                          var,m=M,n=N,size=s) - F(
                                1, b(perm=(1,), fdr=alpha,m=M)[1],null=null_test,fdr=alpha,delta=delt,variance=
                                          var,m=M,n=N,size=s)

0.99

Note: since b[e+1]=1, F(i,b[e+1])=prob(ith hypothesis' p-value<=1)=1 for all i. b[e] is usually very small since it's rejection boundary under BH procedure. This makes all F(i,b[e+1])-F(i,b[e]) very close to 1 thus the results of formula 12(function f_ordinal) is always larger than one for all i and all perm.